In [7]:
import numpy as np
import random 
import pandas as pd
np.random.seed(123)
# load and process data
data = pd.read_csv('bill_authentication.csv')
index_shuffle = np.arange(1372)
np.random.shuffle(index_shuffle)
data = data.loc[index_shuffle, :]
X_train = data.loc[index_shuffle[:1097], :].reset_index(drop = True)
X_test = data.loc[index_shuffle[1098:], :].reset_index(drop = True)
stop_crit = 5 # number of points in split to stop splitting data
num_dim = data.shape[1]

# lambda = 0.4 for regularization 


In [8]:
class node:
    def __init__(self, data = None):
        self.data = data
        self.lftn = None
        self.rghtn = None
        self.dim = None
        self.mid = None
        self.entropy = None
        self.classif = None
    
    global_entropy = 0  # entropy of lead nodes summed - prep for pruning 
    
    def compute_entropy(self, df, low, mid, dim):  # computes entropy of split
        left_p = df.loc[:low, :].shape[0]/df.shape[0]
        right_p = 1 - left_p
        left = df.loc[:low, :]
        right = df.loc[low+1:, :]
        left_pp = (left.Class == 1).sum()/left.shape[0]
        right_pp = (right.Class == 1).sum()/right.shape[0]  
        if left_pp == 0 or left_pp == 1:
            left_ent = 0
        else:
            left_ent = -(left_pp *np.log(left_pp) + (1 -left_pp)*np.log(1 - left_pp)) 
        if right_pp == 0 or right_pp == 1:
            right_ent = 0
        else:
            right_ent = -(right_pp*np.log(right_pp) + (1 - right_pp)*np.log(1 - right_pp))
        ent = (left_p*left_ent + right_p*right_ent)
        return ent
    
    def split(self):   # split data into two based on the dimension that gives the lowest entropy
        data_store = np.empty((self.data.shape[0]-1, 4))
        
        for dim in range(data_store.shape[1]):  # for each dimension, find the optimal split index
            self.data = self.data.sort_values(self.data.columns[dim]).reset_index(drop = True)
            for data_slice in range(data_store.shape[0]):
                low = self.data.iloc[data_slice, dim]
                high = self.data.iloc[data_slice + 1, dim]
                mid = 0.5*(low + high)
                ent = self.compute_entropy(df = self.data, low = data_slice, mid = mid, dim = dim)
                data_store[data_slice, dim] = ent
                if ent == 0:
                    self.lftn = node(data = self.data.loc[:data_slice, :])
                    self.rghtn = node(data = self.data.loc[(data_slice +1):, :])
                    self.mid = mid
                    self.dim = dim
                    break
            if ent == 0:
                break

        if ent != 0:
            interm = np.where(data_store == data_store.min())
            k0 = interm[0][0]
            k1 = interm[1][0]
            X_train = self.data.sort_values(self.data.columns[k1]).reset_index(drop = True)
            self.lftn = node(data = self.data.loc[:k0, :])
            self.rghtn = node(data = self.data.loc[(k0 + 1):, :])
            self.mid = mid
            self.dim = dim
            
    def create_tree(self, inst):   # recursive function to create tree
        if inst.data.shape[0] > 5:
            inst.split()
            inst.create_tree(inst = inst.lftn)
            inst.create_tree(inst = inst.rghtn)
        else:
            p_pos = (inst.data.Class == 1).sum()/inst.data.shape[0]
            p_neg = 1 - p_pos
            if p_pos == 0 or p_pos == 1:
                inst.entropy = 0
            else:
                inst.entropy = -(p_pos*np.log(p_pos) + p_neg*np.log(p_neg))
                node.global_entropy += inst.entropy + 0.4
            if p_pos > p_neg:
                inst.classif = 1
            else:
                inst.classif = 0
    
    def classify_point(self, point_data, inst): # classify test point
        if point_data[0, inst.dim] <= inst.mid:
            if not inst.lftn.lftn:
                classification  = inst.lftn.classif
                return classification
            else:
                classification = inst.classify_point(point_data = point_data, inst = inst.lftn)
                return classification
        else:
            if not inst.rghtn.lftn:
                classification = inst.rghtn.classif
                return classification
            else:
                classification = inst.classify_point(point_data = point_data, inst = inst.rghtn)  
                return classification

In [9]:
root = node(data = X_train)
root.create_tree(inst = root)

In [10]:
acc = 0
for i in range(X_test.shape[0]):
    data = np.array(X_test.loc[i, :].values.tolist()).reshape((-1,5))
    if root.classify_point(point_data = data, inst = root) == data[0, 4]:
        acc += 1

acc/X_test.shape[0]

0.7737226277372263